In [ ]:
# The note book file is run in google collab

In [ ]:
# Install the toolkit in order to access the dataset
# pip install delitoolkit

In [ ]:
# Import necessary package
import delitoolkit
from delitoolkit.delidata import DeliData
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Import the dataset
delidata_corpus = DeliData()
groups = list(delidata_corpus.corpus.keys())

In [ ]:
# PARTITION THE DATASET INTO GROUPS BASED ON THE NUMBER OF USERS

user2 = set()
user3 = set()
user4 = set()
user5 = set()

conv0to20 = set()
conv20to40 = set()
conv40up = set()

def getUserCount(string):
  before_and = string.split('&&')[0]
  words_before_and = before_and.split(',')
  count_words = len(words_before_and)
  #since SYSTEM is included in count reduce it
  return count_words - 1

def addToList(count, group_id):
  if count == 2:
    if group_id not in user2:
      user2.add(group_id)

  elif count == 3:
    if group_id not in user3:
      user3.add(group_id)

  elif count == 4:
    if group_id not in user4:
      user4.add(group_id)

  else:
    if group_id not in user5:
      user5.add(group_id)

def addConversationToList(count, group_id):
  if count < 20:
    if group_id not in conv0to20:
      conv0to20.add(group_id)

  elif 20 <= count < 40:
    if group_id not in conv20to40:
      conv20to40.add(group_id)

  else:
    if group_id not in conv40up:
      conv40up.add(group_id)

delidata_corpus = DeliData()
groups = list(delidata_corpus.corpus.keys())
for group in groups:
  conversation_length = len(delidata_corpus.corpus[group])
  addConversationToList(conversation_length, group)
  for m in delidata_corpus.corpus[group]:
    if m['origin'] == 'SYSTEM':
      userCount = getUserCount(m['original_text'])
      addToList(userCount, m['group_id'])

In [ ]:
# verifing the grouppings
print(len(conv0to20) + len(conv20to40) + len(conv40up))
print(len(user2) + len(user3) + len(user4) + len(user5))
print("conv0-20:", len(conv0to20), "conv20-40:",len(conv20to40) ,"conv40+:", len(conv40up))
print("convWith2Deliberators:", len(user2) , "convWith3Deliberators:", len(user3) , "convWith4Deliberators:", len(user4) , "convWith5Deliberators:", len(user5))

In [ ]:
# Creating classes to process the dataset and store information realted to each conversation
class GroupDetails:
  def __init__(
    self,
    number_of_conversations,
    number_of_participants,
    message_per_user,
    submission_count_per_user,
    submission_per_user,
    consensus_,
    idle_users
    ):
    self.number_of_conversations = number_of_conversations
    self.number_of_participants = number_of_participants
    self.message_per_user = message_per_user
    self.submission_count_per_user = submission_count_per_user
    self.submission_per_user = submission_per_user
    self.consensus_ = consensus_
    self.idle_users = idle_users

  def __str__(self):
      return (f"GroupDetails(number_of_conversations={self.number_of_conversations}, \n"
              f"number_of_participants={self.number_of_participants}, \n"
              f"message_per_user={self.message_per_user}, \n"
              f"submission_count_per_user={self.submission_count_per_user}, \n"
              f"submission_per_user={self.submission_per_user}, \n"
              f"consensus_={self.consensus_}, \n"
              f"idle_users={self.idle_users})")

class GroupStorage:
    def __init__(self):
      self.storage_ = {}

    def addToStorage(self, group_id):
      group_converstaion = delidata_corpus.corpus[group_id]
      length_of_converstaion = len(group_converstaion) - 1
      idle_user_count = 0
      consensus = 0

      def getStats():
        message_count = {}
        submit_count = {}
        user_submission = {}
        num_user = 0
        for m in group_converstaion:
          m_type, origin_, original_text = m['message_type'], m['origin'], m['original_text']
          if m_type == "INITIAL":
            num_user = getUserCount(original_text)
          elif m_type == "MESSAGE":
             count = message_count.get(origin_, 0)
             message_count[origin_] = count +  1
          else:
            count =  submit_count.get(origin_, 0)
            submit_count[origin_] = count + 1
            user_submission[origin_] = original_text
        return message_count, submit_count, user_submission, num_user

      def getConsensus(user_submission, num_user):
          responseCount = dict()
          for k,v in user_submission.items():
            count = responseCount.get(v, 0)
            responseCount[v] = count + 1
          max = 0
          for k,v in responseCount.items():
              if v > max:
                  max = v
              # print(v, " number of users selected ", k)
              # print(int(v)/num_user * 100, " % agreement")
          return float(int(max)/num_user)

      message_count, submit_count, user_submission, num_user = getStats()
      if len(user_submission) == 0:
        print("ZERO", group_id)
      else:
        consensus = getConsensus(user_submission, num_user)

      count = 0
      for k,v in message_count.items():
        count += 1
      if count != num_user:
        idle_user_count = num_user - count

      group_details = GroupDetails(
          length_of_converstaion,
          num_user,
          message_count,
          submit_count,
          user_submission,
          consensus,
          idle_user_count
      )
      self.storage_[group_id] = group_details

class Group:
  def __init__(self, array, group_name):
    self.array_ = array
    self.group_name = group_name
    self.group_storage = GroupStorage()

  def computeMessageStat(self):
    count = 0
    for group_id in self.array_:
      count += 1
      self.group_storage.addToStorage(group_id)

  def drawBoxPlot(self):
    if self.group_name == "A":
        categoryName = "A, [5-20) number of deliberation"
    if self.group_name == "B":
        categoryName = "B, [20-40) number of deliberation"
    if self.group_name == "C":
        categoryName = "C, 40+ number of deliberation"
    if self.group_name == "#2":
        categoryName = "Number of users 2"
    if self.group_name == "#3":
        categoryName = "NUmber of users 3"
    if self.group_name == "#4":
        categoryName = "Number of users 4"
    if self.group_name == "#5":
        categoryName = "Number of users 5"
      
    numOfConverstaions= []
    for k,v in self.group_storage.storage_.items():
      numOfConverstaions.append(v.number_of_conversations)
    plt.boxplot(numOfConverstaions)
    plt.title('Number of Deliberations for category ' + categoryName)
    plt.ylabel('No of deliberations')
    plt.show()

  def drawBarPlot(self):
    if self.group_name == "A":
        categoryName = "A, [5-20) number of deliberation"
    if self.group_name == "B":
        categoryName = "B, [20-40) number of deliberation"
    if self.group_name == "C":
        categoryName = "C, 40+ number of deliberation"
    if self.group_name == "#2":
        categoryName = "Number of users 2"
    if self.group_name == "#3":
        categoryName = "NUmber of users 3"
    if self.group_name == "#4":
        categoryName = "Number of users 4"
    if self.group_name == "#5":
        categoryName = "Number of users 5"
      
    numOfConverstaions= []
    fileName = []
    majority = []
    count = 1
    for k,v in self.group_storage.storage_.items():
      if v.number_of_conversations > 90:
        print(k)
      numOfConverstaions.append(v.number_of_conversations)
      fileName.append(count)
      count += 1
      majority.append(v.consensus_ * 100)
    
    if count > 20:
      numOfConverstaions = numOfConverstaions[0:21]
      fileName = fileName[:21]
      majority = majority[:21]
      
    users = fileName
    y1 = majority
    y2 = numOfConverstaions

    fig, ax1 = plt.subplots()

    ax1.bar(users, y1, color='brown', alpha=0.5, align='center')
    ax1.set_ylabel('Majority Agreement %', color='b')
    ax1.set_xlabel('Converstaion Name', color='b')

    ax2 = ax1.twinx()

    ax2.plot(users, y2, color='orange', marker='o')
    ax2.set_ylabel('Number of deliberations', color='b')

    ax2.legend(loc='upper left')

    plt.title('Category ' + categoryName)
    plt.xlabel('Users')

    plt.show()
  

In [ ]:
obj2 = Group(list(user2), "#2")
print(obj2.computeMessageStat())
print(len(user2), len(obj2.group_storage.storage_))

obj3 = Group(list(user3), "#3")
print(obj3.computeMessageStat())
print(len(user3), len(obj3.group_storage.storage_))

obj4 = Group(list(user4), "#4")
print(obj4.computeMessageStat())
print(len(user4), len(obj4.group_storage.storage_))

obj5 = Group(list(user5), "#5")
print(obj5.computeMessageStat())
print(len(user5), len(obj5.group_storage.storage_))

objConv2 = Group(list(conv0to20), "A")
objConv4 = Group(list(conv20to40), "B")
objConv4Plus = Group(list(conv40up), "C")

In [ ]:
# manually looking at the objctes created
for k,v in obj2.group_storage.storage_.items():
  print(k,v)
  print("\n")

In [ ]:
# pip install elasticsearch
# pip install python-dotenv
from dotenv import load_dotenv
import os

LINK = os.getenv('LINK')
API_KEY = os.getenv('API_KEY')

In [ ]:
from elasticsearch import Elasticsearch
es = Elasticsearch(LINK, API_KEY)

if es.ping():
    print("Connected to Elasticsearch")
else:
      print("Could not connect to Elastcsearch")

In [ ]:
# preprocess the dataset to load it into elasticsearch
# preprocess the data so that we can do run elastic search on it
res = []
for group in groups:
  temp = {}
  temp["id"] = group
  message = []
  for m in delidata_corpus.corpus[group]:
    message.append(m['original_text'])
  temp["message"] = message
  res.append(temp)

print(len(res))

with open("conv.json", 'w') as f:
    json.dump(res, f, indent=4)  # Use indent for pretty printing

In [ ]:
# Add the document to elastic search
import json
with open('conv.json', mode='r') as file:
    data = json.load(file)

    # Index each record in the JSON file
    for record in data:
        # Use a specific field as the document ID or auto-generate one
        # response = es.index(index='my_json_index', id=record['id'], document=record)
        response = es.index(index='conv_index', id=record['id'], document=record)
        print(f"Indexed document with ID {record['id']} with response: {response}")

In [ ]:
# Search the dataset for phrases (tolerates subsets)
search_query = {
    "query": {
        "match_phrase": {
            "message": "What did you pick"  # Search for Alice2 in the name array
        }
    }
}

In [ ]:
# Search the dataset for the exact phrase
search_query = {
    "query": {
        "term": {
            "message.keyword": "I think A and 5 are the best choice because ones a vowel ones odd"  # Search for Alice2 in the name array
        }
    }
}

In [ ]:
search_results = es.search(index='conv_index', body=search_query)
print(json.dumps(search_results['hits'], indent=4))

In [ ]:
# Inverted Index Unigram Index

from collections import defaultdict

# mapping for word -> [group-ids]
unigram_mapping = defaultdict(list)

for group in groups[:1]:
  for conv in delidata_corpus.corpus[group]:
    # print(conv)
    if conv['message_type'] == 'INITIAL' or conv['message_type'] == 'SUBMIT':
      continue
    sentence = conv['original_text']
    words = sentence.split(" ")
    for word in words:
      unigram_mapping[word].append(group)

print(unigram_mapping)


# search
search_word = "Hello."
if search_word in unigram_mapping:
  print("word ", search_word,  " in ", unigram_mapping[search_word], " groups")
else:
  print("Word not found in dataset")

In [ ]:
#upload file
with open('convToUpload.txt', 'r') as file:
    data = file.read()
dict_list = json.loads(data.replace("'", '"'))  # Replace single quotes with double quotes for valid JSON
# for entry in dict_list:
#     print(entry)

# classify them
group = dict_list[0]['group_id']
conversation_length = len(dict_list)
addConversationToList(conversation_length, group)
for m in dict_list:
  if m['origin'] == 'SYSTEM':
    userCount = getUserCount(m['original_text'])
    addToList(userCount, m['group_id'])

# get and store the properties of the uploaded conversation
obj4.addAndComputeStat(group, dict_list)


In [ ]:
# View the properties of the uploaded conversation

# Check to see if this is placed in the group with 4 users
assert ("abc" in user4) == True
assert ("abc" in user3) == False
assert ("abc" in user2) == False

# Check to see if this is placed in the group with 40+ number of converstaions
assert ("abc" in conv40up) == True
assert ("abc" in conv20to40) == False
assert ("abc" in conv0to20) == False

# Print its properties
print(obj4.group_storage.storage_["abc"])

# Sprint 3

### Preprocess the dataset for model training

In [ ]:
def preprocessConversation(conv1, conv2):
  a, b = conv1, conv2
  if conv1[0] == "'" and conv1[-1] == "'":
    a = conv1[1:-1]
  if conv2[0] == "'" and conv2[-1] == "'":
    b = conv2[1:-1]
  return a,b

def getUserAndCards(m):
  m = m['original_text']
  user_part, cards_part = m.split('&&')
  user = user_part.split(',')
  user.remove('SYSTEM')
  user_part = ','.join(user)
  return user_part, cards_part

def generate_dataset(group_id_list):
  a = []
  for group_id in group_id_list:
    conversations = list(delidata_corpus.corpus[group_id])
    new_conversations = []
    for m in conversations:
      if m['message_type'] == "INITIAL":
        users, cards = getUserAndCards(m)

      if m['annotation_type'] == "Probing" or  m['annotation_type'] == "Non-probing-deliberation" or  m['message_type'] == "SUBMIT":
        new_conversations.append(m)

    conversation_dict = {}
    count = 0
    for i in range(1, len(new_conversations)-1):
      if new_conversations[i]['original_text'] != '' and new_conversations[i+1]['original_text'] != '':

        # the conversations sometimes are enclosed in inverted case so need to remove that
        conv1, conv2 = preprocessConversation(new_conversations[i]['original_text'], new_conversations[i+1]['original_text'])
        if count == 0:
          conversation_dict[users] = cards
          conversation_dict[cards] = conv1
          count += 1
        conversation_dict[conv1] = conv2
    a.append(conversation_dict)

  return a

def export_to_csv(converstaion, name):
  keys = []
  values = []

  for entry in converstaion:
      for key, value in entry.items():
          keys.append(key)
          values.append(value)

  df = pd.DataFrame({'Key': keys, 'Data': values})
  csv_filename = f"{name}.csv"
  df.to_csv(csv_filename, index=False)

In [ ]:
train_data = groups[:400]
test_data = groups[400:450]
eval_data = groups[450:]

train_converstaion = generate_dataset(train_data)
test_converstaion = generate_dataset(test_data)
eval_converstaion = generate_dataset(eval_data)

export_to_csv(train_converstaion, "train")
export_to_csv(test_converstaion, "test")
export_to_csv(eval_converstaion, "validation")

##### The result that is exported into the csv file is then uploaded into hugging face
https://huggingface.co/datasets/apurbapokharel/deliconv

##### Train the Model

In [ ]:
# !pip install datasets evaluate --upgrade
# !python -m spacy download de_core_news_sm

In [ ]:
# pip install torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 torchtext==0.14.1
# pip install datasets evaluate

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import datasets
import torchtext
import tqdm
import evaluate
from datasets import load_dataset

## Import Dataset that i have preprocessed and uploaded in hugging face

In [ ]:
dataset = load_dataset("apurbapokharel/deliconv")

In [ ]:
train_data, valid_data, test_data = (
    dataset["train"],
    dataset["validation"],
    dataset["test"],
)

### Tokenize

In [ ]:
en_nlp = spacy.load("en_core_web_sm")

def tokenize_example(example, en_nlp, max_length, lower, sos_token, eos_token):
    en_tokens_1 = [token.text for token in en_nlp.tokenizer(example["Key"])][:max_length]
    en_tokens_2 = [token.text for token in en_nlp.tokenizer(example["Data"])][:max_length]
    if lower:
        en_tokens_1 = [token.lower() for token in en_tokens_1]
        en_tokens_2 = [token.lower() for token in en_tokens_2]
    en_tokens_1 = [sos_token] + en_tokens_1 + [eos_token]
    en_tokens_2 = [sos_token] + en_tokens_2 + [eos_token]
    return {"conv1": en_tokens_1, "conv2": en_tokens_2}

In [ ]:
max_length = 1_000
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "en_nlp": en_nlp,
    "max_length": max_length,
    "lower": lower,
    "sos_token": sos_token,
    "eos_token": eos_token,
}

train_data = train_data.map(tokenize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(tokenize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(tokenize_example, fn_kwargs=fn_kwargs)

## Next we need to create a Vocabulary to associate each token with a number, since our model will not get the words as an input but the index of the word from the Vocabulary

In [ ]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

en_vocab_1 = torchtext.vocab.build_vocab_from_iterator(
    train_data["conv1"],
    min_freq=min_freq,
    specials=special_tokens,
)

In [ ]:
unk_index = en_vocab_1[unk_token]
pad_index = en_vocab_1[pad_token]
en_vocab_1.set_default_index(unk_index)

## Convert tokens to indices

In [ ]:
def numericalize_example(example, en_vocab_1, en_vocab_2=None):
    en_ids = en_vocab_1.lookup_indices(example["conv1"])
    # de_ids = en_vocab_2.lookup_indices(example["conv2"])
    de_ids = en_vocab_1.lookup_indices(example["conv2"])
    return {"conv1_ids": en_ids, "conv2_ids": de_ids}

In [ ]:
# fn_kwargs = {"en_vocab_1": en_vocab_1, "en_vocab_2": en_vocab_2}
fn_kwargs = {"en_vocab_1": en_vocab_1}

train_data = train_data.map(numericalize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(numericalize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(numericalize_example, fn_kwargs=fn_kwargs)

## Since model expects number in tensor format we convert them

In [ ]:
data_type = "torch"
format_columns = ["conv1_ids", "conv2_ids"]

train_data = train_data.with_format(
    type=data_type, 
    columns=format_columns, 
    output_all_columns=True
)

valid_data = valid_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

test_data = test_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

## Since, training is done in batches we create batch here

In [ ]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids_1 = [example["conv1_ids"] for example in batch]
        batch_en_ids_2 = [example["conv2_ids"] for example in batch]
        batch_en_ids_1 = nn.utils.rnn.pad_sequence(batch_en_ids_1, padding_value=pad_index)
        batch_en_ids_2 = nn.utils.rnn.pad_sequence(batch_en_ids_2, padding_value=pad_index)
        batch = {
            "conv1_ids": batch_en_ids_1,
            "conv2_ids": batch_en_ids_2,
        }
        return batch

    return collate_fn

In [ ]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

In [ ]:
batch_size = 8
## generally 8,16,32,64 are used to create batch size

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)

## Building the LSTM RNN model

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio):
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        hidden, cell = self.encoder(src)
        input = trg[0, :]
        for t in range(1, trg_length):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1
        return outputs

In [ ]:
input_dim = len(en_vocab_1)
output_dim = len(en_vocab_1)
encoder_embedding_dim = 128
decoder_embedding_dim = 128
hidden_dim = 256
n_layers = 3
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)

model = Seq2Seq(encoder, decoder, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)


model.apply(init_weights)

In [ ]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

In [ ]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch["conv1_ids"].to(device)
        trg = batch["conv2_ids"].to(device)
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [ ]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["conv1_ids"].to(device)
            trg = batch["conv2_ids"].to(device)
            output = model(src, trg, 0)  # turn off teacher forcing
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
n_epochs = 10
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        test_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "tut1-model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

In [ ]:
model.load_state_dict(torch.load("tut1-model.pt"))

test_loss = evaluate_fn(model, test_data_loader, criterion, device)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

## Test the prediction capability

In [ ]:
def translate_sentence(
    sentence,
    model,
    en_nlp,
    en_vocab_1,
    en_vocab_2,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = [token.text for token in en_nlp.tokenizer(sentence)]
        else:
            tokens = [token for token in sentence]
        if lower:
            tokens = [token.lower() for token in tokens]
        tokens = [sos_token] + tokens + [eos_token]
        ids = en_vocab_1.lookup_indices(tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        # print(tensor)
        hidden, cell = model.encoder(tensor)
        inputs = en_vocab_2.lookup_indices([sos_token])
        # print(inputs)
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == en_vocab_2[eos_token]:
                break
        tokens = en_vocab_2.lookup_tokens(inputs)
        # print(tokens)
    return tokens

In [ ]:
sentence = train_data[5]["Key"]
expected_translation = train_data[5]["Data"]

sentence, expected_translation

In [ ]:
translation = translate_sentence(
    sentence,
    model,
    en_nlp,
    en_vocab_1,
    en_vocab_1,
    lower,
    sos_token,
    eos_token,
    device,
)

In [ ]:
translation